# Parsing Infrastrucutre Vulnerability Scans

In [14]:
# Import Libraries
import mariadb as db
import xml.etree.ElementTree as et
import requests
import sys
##from os import getcwd
##import io

In [15]:
url = "https://raw.githubusercontent.com/DefectDojo/sample-scan-files/master/nessus/nessus_v_unknown.nessus"

download = requests.get(url).text

tree = et.fromstring(download)

In [122]:
data = []
for host in tree.findall('Report/ReportHost'):
    ipaddr = host.find("HostProperties/tag/[@name='host-ip']").text

    for item in host.findall('ReportItem'):
        risk_factor = item.find('risk_factor').text
        pluginID = item.get('pluginID')
        pluginName = item.get('pluginName')
        port = item.get('port')
        protocol = item.get('protocol')  
        data.append(
          (ipaddr,
          risk_factor,
          protocol+" "+port,
          pluginID,
          pluginName)
        )
print(data[0:5])

[('10.31.112.29', 'None', 'tcp 0', '19506', 'Nessus Scan Information'), ('10.31.112.29', 'None', 'tcp 0', '10919', 'Open Port Re-check'), ('10.31.112.29', 'Medium', 'tcp 3389', '58453', "Terminal Services Doesn't Use Network Level Authentication (NLA)"), ('10.31.112.29', 'None', 'tcp 0', '54615', 'Device Type'), ('10.31.112.29', 'None', 'tcp 0', '45590', 'Common Platform Enumeration (CPE)')]


In [120]:
try:
    conn = db.connect(
        user="databunker",
        password="checkoutmystrongpassword",
        host="localhost",
        port=3306,
        database ="riskrptdb"
    )
except db.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)
    
cur = conn.cursor()

In [123]:
table_name = "nessus_sample"
params = (table_name,)
columns = """
    ip%s,
    risk%s,
    port%s,
    pluginID%s,
    pluginName%s
    """
drop = ("DROP TABLE IF EXISTS %s" %params)
create = ("CREATE TABLE %s" %params)
insert = ("INSERT INTO %s "%params) + "(ip, risk, port, pluginID, pluginName) VALUES (%s, %s, %s, %s, %s)"

with conn.cursor() as cursor:
    cursor.execute(drop)
    cursor.execute(create + "(" + columns % (' VARCHAR(50)',' VARCHAR(24)',' VARCHAR(50)',' VARCHAR(15)',' VARCHAR(256)',)+",rowID int PRIMARY KEY AUTO_INCREMENT)")
    cursor.executemany(insert, data)
conn.commit()